In [128]:
!kaggle competitions download -c spaceship-titanic

HTTPSConnectionPool(host='storage.googleapis.com', port=443): Max retries exceeded with url: /kaggle-competitions-data/kaggle-v2/34377/3220602/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1766610803&Signature=A5%2BZNqqTb0LkSc%2FZGAY2kLNr635bSzWqV1rTFiTc6df8SYP%2FgoXNS5htZIhv7K86xHQsjom22GfEEJO%2Bj1oRhbj9TpwNCLHQXs0I57xseCjIcKYvtKUBvUbmcuLDS9OLQlDO3r%2Fuga6nQjFAX6ZLB%2B%2B9h4mYwRpnbEelUkGFf%2BFhrdBARpeua8jResl7ZcLR59thVeGGf5DIeeoR9U%2Flr%2BKhhMawnwlgLoaOkgzQmSzrG1HvikUBpAqmJ96%2FY4MDbzdpWwDOyPaZoM5sc6Vd8dlwixKItLlJc2PuBiaOqZpZaDdjCLpjsPFsl2Zgdmg2ORVACOSuSGzzBtzdapW3Uw%3D%3D&response-content-disposition=attachment%3B+filename%3Dspaceship-titanic.zip (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1147)')))


In [129]:
import zipfile
import os
import pandas as pd

# 解压数据集
path = "spaceship-titanic"
if not os.path.exists(path=path):
    os.makedirs(name=path)
fp = zipfile.ZipFile("spaceship-titanic.zip", 'r')
fp.extractall(path)

# 读取数据
train_csv = pd.read_csv(filepath_or_buffer=f"{path}/train.csv")
tests_csv = pd.read_csv(filepath_or_buffer=f"{path}/test.csv")
train_csv

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [130]:
# 初步组合数据
# 分析特征-Name：游客名称，姓名特征对预测没有影响，删除处理
all_features = pd.concat((train_csv.iloc[:, 0:-2], tests_csv.iloc[:, 0:-1])) # 删除name和labels以及PassengerId等无关特征
all_features = all_features.reset_index(drop=True) # 重置索引避免重复
all_features[1:3]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0


In [131]:
import matplotlib.pyplot as plt
from pandas import Series

def groupby_cate(data: Series) -> dict[str, float]:
    """统计离散数据频数

    Args:
        data (list): _description_

    Returns:
        dict[str, float]: _description_
    """
    cate_frequent = {}
    for cate in data:
        if cate not in cate_frequent:
            cate_frequent[cate] = 1
        else:
            cate_frequent[cate] = cate_frequent[cate] + 1
    return cate_frequent

In [132]:
# 分析特征-HomePlanet 将nan设置为Earth 采用one-hot encoding处理本列数据
all_features['HomePlanet'] = all_features['HomePlanet'].fillna('Earth')
print(groupby_cate(all_features['HomePlanet']))

{'Europa': 3133, 'Earth': 7153, 'Mars': 2684}


In [133]:
# 分析特征-CryoSleep（处理NaN，存在消费的CryoSleep一定是False，另外被限制在船舱内的自救难度可能增加）
for index, row in all_features[all_features['CryoSleep'].isna() == True].iterrows():
    if row.RoomService > 0 or row.FoodCourt > 0 or row.ShoppingMall > 0 or row.Spa > 0 or row.VRDeck > 0:
        all_features.loc[index, 'CryoSleep'] = False
    else:
        all_features.loc[index, 'CryoSleep'] = False
all_features['CryoSleep'] = all_features['CryoSleep'].astype(int)
all_features[all_features['CryoSleep'].isna() == True]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck


In [134]:
# 分析特征-Cabin：客舱，乘客入住的客舱编号，格式为甲板/编号/侧、P表示左舷、S表示右舷
# 分解特征数据创建新的特征，分别是甲板、编号（不作为数字）、侧进行one-hot encoding
all_features['Cabin'].value_counts()
all_features[['Deck', 'Num', 'Side']] = all_features['Cabin'].str.split('/', expand=True)
del all_features['Cabin']
all_features[1:3]

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
1,0002_01,Earth,0,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,0,S
2,0003_01,Europa,0,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,0,S


In [135]:
# 分析特征-Destination、Age，前者缺失值使用众数补充，后者使用均值处理
all_features['Age'] = all_features['Age'].fillna(all_features['Age'].mean())
all_features['Age']

0        39.000000
1        24.000000
2        58.000000
3        33.000000
4        16.000000
           ...    
12965    34.000000
12966    42.000000
12967    28.771969
12968    28.771969
12969    43.000000
Name: Age, Length: 12970, dtype: float64

In [136]:
all_features['Destination'] = all_features['Destination'].fillna('TRAPPIST-1e')
all_features['Destination'].value_counts()

Destination
TRAPPIST-1e      9145
55 Cancri e      2641
PSO J318.5-22    1184
Name: count, dtype: int64

In [137]:
# 分析特征-VIP，使用众数填充
all_features['VIP'] = all_features['VIP'].fillna(False)
all_features['VIP'].value_counts()
all_features['VIP'] = all_features['VIP'].astype(int)

In [138]:
# 分析特征-RoomService、FoodCount、ShoppingMall、Spa、VRDeck等消费特征对获救无关并且大多数为0，可先考虑删除处理，若对分数存在影响则进行标准化处理
del all_features['RoomService']
del all_features['FoodCourt']
del all_features['ShoppingMall']
del all_features['Spa']
del all_features['VRDeck']
all_features[1:3]

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,Deck,Num,Side
1,0002_01,Earth,0,TRAPPIST-1e,24.0,0,F,0,S
2,0003_01,Europa,0,TRAPPIST-1e,58.0,1,A,0,S


In [139]:
# 分析特征-PassengerId，其中包含了旅行团信息，是否在旅行团中可能对获救存在影响，可分解旅行团，并创建是否抱团特征
all_features['GroupID'] = all_features['PassengerId'].str.split('_').str[0]
all_features['IsAlone'] = (all_features.groupby('GroupID').transform('size') == 1).astype(int)
del all_features['GroupID']
del all_features['PassengerId']
all_features[1:3]

,HomePlanet,CryoSleep,Destination,Age,VIP,Deck,Num,Side,IsAlone
1,Earth,0,TRAPPIST-1e,24.0,0,F,0,S,1
2,Europa,0,TRAPPIST-1e,58.0,1,A,0,S,0
